In [14]:
import pandas as pd
import requests
import json
import re
import time
import os
import csv
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [60]:
class_types = ['fine-art',
               'photography',
               'graphic-design',
               'illustration',
               'writing',
               'music-production',
               'animation',
               'ui-ux-design',
               'film-production',
               'marketing',
               'entrepreneurship',
               'productivity',
               'finance',
               'freelance',
               'business-analytics',
               'management',
               'leadership',
               'sales',
               'human-resources',
               'accounting',
               'web-development',
               'mobile-development',
               'it-security',
               'data-science',
               'game-design',
               'product-management',
               'crafts',
               'culinary',
               'health-and-wellness',
               'other',
               'teaching',
               'home-business',
               'languages',
               'gaming']


In [4]:
def make_url(class_type_list):
    urls_dict = {}
    url_template = 'https://www.skillshare.com/browse/%?sort=rating&seeAll=1'
    for class_type in class_type_list:
        urls_dict[class_type] = url_template.replace('%', class_type)
    return urls_dict

urls_dict = make_url(class_types)

with open('class_list_urls.csv', 'w') as f:
    writer = csv.writer(f)
    for key, val in urls_dict.items():
        writer.writerow([key, val])


In [5]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

class_links_dict = {}

In [7]:
def get_class_link(class_types, urls_dict):
    for class_type in class_types:
        i=0
        print('Working on getting list of ' + class_type + ' classes')
        url = urls_dict[class_type]
        
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)

        try:
            close = driver.find_element('xpath', "//a[contains(@class,'btn-close ss-icon-close')]")
            close.click()
        except:
            pass

        lastHeight = driver.execute_script("return document.body.scrollHeight")
        while True:
            i+=1
            print("Load ", i)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(120)
            newHeight = driver.execute_script("return document.body.scrollHeight")
            if newHeight == lastHeight:
                break
            lastHeight = newHeight

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        class_link_tags = soup.find_all('div', attrs={"class": "col-4 class-column rendered "})
        class_links = [class_link_tag.find('p').find('a')['href'] for class_link_tag in class_link_tags]
        class_links_dict[class_type] = class_links
        driver.close()
        
get_class_link(class_types, urls_dict)

with open('class_urls.csv', 'w') as f:
    writer = csv.writer(f)
    for key, val in class_links_dict.items():
        writer.writerow([key, val])

Working on getting list of accounting classes
Load  1
Load  2


In [ ]:
driver.close()

In [200]:
def get_links_from_txt(file):
    with open(file) as f:
        urls_dict = {}
        
        reader = f.readlines()
        
        for line in reader:
            isStart = True
            foo = line.strip().split(',')
            for item in foo:
                clean_item = item.strip().strip("[").strip("]").strip("'").strip()
                if isStart:
                    isStart = False
                    class_type = clean_item
                    urls_dict[class_type] = []
                else:
                    urls_dict[class_type].append(clean_item)
        return urls_dict
        
file = 'class_urls-1.csv'
urls_dict = get_links_from_txt(file)

In [202]:
urls_dict.keys()

dict_keys(['fine-art', 'photography', 'graphic-design', 'illustration', 'writing', 'music-production', 'animation', 'ui-ux-design', 'film-production', 'marketing', 'entrepreneurship', 'productivity', 'finance', 'freelance', 'business-analytics', 'management', 'leadership', 'sales', 'human-resources', 'accounting', 'web-development', 'mobile-development', 'it-security', 'data-science', 'game-design', 'product-management', 'crafts', 'culinary', 'health-and-wellness', 'other', 'teaching', 'home-business', 'languages', 'gaming'])

In [125]:
test_case = 'https://www.skillshare.com/classes/Create-a-Perfect-Morning-Routine/287216370?via=browse-rating-health-and-wellness-layout-grid'
response = requests.get(test_case)
page = response.text
soup = BeautifulSoup(page, "lxml")
get_title_and_author(soup)
get_class_description(soup)
get_project_authors(soup)
get_tags(soup)
get_video_length(soup)

'17 Videos (34m)'

In [205]:
import random


for class_type in class_types:
    foo = random.randint(5,10)
    test_cases = urls_dict[class_type][:foo]
    for test_case in test_cases:
        response = requests.get(test_case)
        page = response.text
        soup = BeautifulSoup(page, "lxml")

        javascript_data = get_javascript_data(soup)

        print(get_title_and_author(soup))
        print(get_detial_author_info(javascript_data))
        print(get_class_description(soup))    
        print(get_video_length(soup))
        print(get_tags(soup))
        print(get_project_authors(javascript_data))
        print(get_class_sku(javascript_data))
        print(isPremium(javascript_data))
        print(javascript_data['pageData']['headerData']['tagText'] == 'Premium class')
        print(get_start_date(javascript_data))
        print(get_enrollment_number(javascript_data))

["Modern Watercolor Techniques: Beginner's Level", 'Ana Victoria CalderÃ³n']
('Ana Victoria Calderón', 'Artist', 'https://www.skillshare.com/user/anavictoriana')
(587, 1, 0)
('11', '20')
['Watercolors', 'Fine Art', 'Creative', 'Anavictoriana', 'Ana Victoria', 'Galaxy', 'Universe']
['Ana Victoria Calderón', 'Casey Saccomanno', 'Violeta Samson', 'Denitsa Blagoeva', 'Nelli Khatmoullina', 'Kat Rutt', 'Elena Falovskaia', 'Meda Halmaciu']
576876085
True
True
07/21/2015
18,322
['Ink & Watercolor Magic: 5 Step By Step Illustrations', 'Yasmina Creates']
('Yasmina Creates', 'Ink & Watercolor Artist', 'https://www.skillshare.com/user/yasminacreates')
(1123, 2, 0)
('11', '55')
['Art', 'DIY', 'Illustration', 'Watercolors', 'Drawing', 'Painting', 'Fine Art', 'Creative']
['Scarlet Keen', 'Erin Mears', 'Megi Provencher', 'Zuzana Volny', 'Lucy Lambriex', 'Gordana Brborovic', 'Joanna Hall', 'Diana Nemesu']
1205014536
True
True
09/22/2017
2,013
['Watercolor: 9 Tropical Leaves', 'Peggy Dean']
('Peggy Dean

['Illustrator Secrets: 5 Techniques to Add Realism to Your Work', 'DKNG Studios']
('DKNG Studios', 'Design + Illustration ', 'https://www.skillshare.com/user/dkngstudios')
(1360, 0, 1)
('8', '40')
['Adobe Illustrator', 'Graphic Design', 'Creative', 'Vector Art', 'Vector Graphics']
['Matt Benson', 'Jonathon Kutt', 'Ana-Irina Vescan', 'Connor Goicoechea', 'Miguel Sanchez', 'Adam Pilat', 'Dan Jennings', 'Vicente Sarmento']
1614992610
True
True
08/14/2017
4,671
['Basics of Photoshop: Fundamentals for Beginners', 'Meg Lewis']
('Meg Lewis', 'Designer', 'https://www.skillshare.com/user/darngooood')
(5298, 4, 4)
('16', '18')
['Design', 'Adobe Photoshop', 'Composition', 'Graphic Design', 'Creative']
['Lacie Schoch', 'L B', 'Alyssa Anthony', 'Jayne Travers', 'Janice R', 'John Ryan', 'Christina Mule', 'Diana Chiu']
1320270520
True
True
12/04/2013
19,469
['Graphic Design Basics: Core Principles for Visual Design', 'Ellen Lupton']
('Ellen Lupton', 'Curator, Cooper Hewitt, Smithsonian Design Museum'

['Storytelling Fundamentals: Character, Conflict, Context, Craft', 'Daniel JosÃ© Older']
('Daniel José Older', 'New York Times Bestselling Author', 'https://www.skillshare.com/user/djolder')
(1204, 0, 1)
('9', '42')
['Writing & Publishing', 'Storytelling', 'Writing', 'Short Stories', 'Fiction', 'Creative Writing', 'Creative']
['Daniel José Older', 'Angela Alvarez Velez', 'Chris Stevens', 'Megan Salgado', 'Jay R', 'Catherine Beverage', 'Myra ...', 'April Jones']
690933742
True
True
06/08/2016
5,669
['Creative Writing for All: A 10-Day Journaling Challenge', 'Emily Gould']
('Emily Gould', 'Writer', 'https://www.skillshare.com/profile/Emily-Gould/6846545')
(1300, 0, 1)
('6', '26')
['Writing & Publishing', 'Writing', 'Creative Writing', 'Creative', 'Journaling', 'Creativity', 'Diary']
['Emily Gould', 'Emily K', 'Elaine Fung', 'Beka Cloninger Olson', 'Andia Kolakowski', 'Chris Curro', 'Diane Johnson', 'D George']
1042474131
True
True
03/25/2016
6,998
['Blogging Basics (FREE)', 'Theresa Chri

['Animating With Character - Rigging', 'Jake Bartlett']
('Jake Bartlett', 'Motion Designer', 'https://www.skillshare.com/user/jakebartlett')
(538, 1, 0)
('21', '27')
['Animation', 'Motion Graphics', 'Adobe After Effects', 'Graphic Design', 'Creative']
['Jake Bartlett', 'Ann Rubin', 'Maria Keller', 'Josip Vranjkovic', 'Jihye Jeon', 'Daniel Soto', 'Hayden Aube', 'Jesse Jones']
1435213963
True
True
01/19/2016
4,097
['The Ultimate Guide to Kinetic Type in After Effects', 'Jake Bartlett']
('Jake Bartlett', 'Motion Designer', 'https://www.skillshare.com/user/jakebartlett')
(2157, 3, 1)
('11', '7')
['Design', 'Typography', 'Animation', 'Motion Graphics', 'Adobe After Effects', 'Graphic Design', 'Creative']
['David Fortin', 'Nuria Boj', 'Jeff Madsen', 'João R Zanetti', 'Shannon Snow', 'Robert Headrick', 'Jessie Contour', 'Chris DeCarlo']
282677337
True
True
11/26/2013
8,464
['Move from Graphic Designer to UX DESIGNER', 'Daniel Scott']
('Daniel Scott', 'Adobe Certified Trainer & UI/UX Expert ',

['Create a Short Documentary Video Portrait', 'Elaine McMillion Sheldon']
('Elaine McMillion Sheldon', 'Documentary Storyteller', 'https://www.skillshare.com/user/elainemcsheldon')
(1608, 0, 1)
('11', '54')
['Film', 'Film Production', 'Filmmaking', 'Journalism', 'Short Films', 'Documentary', 'Video Editing', 'Creative']
['Rachman Blake', 'Maiu Lünekund', 'Chaz Langley', 'Lauren Brooks', 'Marcel Czielontko', 'Anniken Haldna']
337067753
True
True
05/04/2016
3,375
['Context is Key: Social Media Strategy in a Noisy Online World', 'Gary Vaynerchuk']
('Gary Vaynerchuk', 'Wine Guru and CEO, Vaynermedia', 'https://www.skillshare.com/user/garyvee')
(2364, 1, 0)
('24', '31')
['Entrepreneurship', 'Marketing', 'Business', 'Digital Strategy', 'Social Media', 'Content Strategy']
['Carlo Powe', 'Josh Schermer', 'Juan Pablo Solano', 'Matej Jan', 'Kelley Fitch', 'Brett Coffey', 'Linda Yi', 'Julie Smith']
101309737
True
True
11/13/2013
20,059
['Analytics & Authenticity on Instagram: Crafting Your Digita

['Visual Storytelling: Creating More Persuasive Presentations', 'Susannah Shattuck']
('Susannah Shattuck', 'Content Marketing Manager at Prezi', 'https://www.skillshare.com/user/susannahshattuck')
(1741, 0, 2)


AttributeError: 'NoneType' object has no attribute 'group'

In [191]:
for test_case in test_cases:
    response = requests.get(test_case)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    javascript_data = get_javascript_data(soup)
    
    print(get_title_and_author(soup))
    print(get_detial_author_info(javascript_data))
    print(get_class_description(soup))    
    print(get_video_length(soup))
    print(get_tags(soup))
    print(get_project_authors(javascript_data))
    print(get_class_sku(javascript_data))
    print(isPremium(javascript_data))
    print(javascript_data['pageData']['headerData']['tagText'] == 'Premium class')
    print(get_start_date(javascript_data))
    print(get_enrollment_number(javascript_data))

["The Writer's Toolkit: 6 Steps to a Successful Writing Habit", 'Simon Van Booy']
('Simon Van Booy', 'Author and Screenwriter', 'https://www.skillshare.com/profile/Simon-Van-Booy/1698932')
(750, 0, 0)
('9', '38')
['Writing & Publishing', 'Writing', 'Creative Writing', 'Creative', 'Journaling', 'Creativity']
['Simon Van Booy', 'Kip Mistral', 'Roger K.', 'Pongsakorn Suppakittpaisarn', 'Catherine Vaughan', 'C.T. Ludgate', 'One of the Jennifers', 'Elaine Hayden-Booker']
1072824603
True
True
08/17/2016
3,346
['Creative Nonfiction: Write Truth with Style', 'Susan Orlean']
('Susan Orlean', 'Staff Writer, The New Yorker', 'https://www.skillshare.com/profile/Susan-Orlean/9873313')
(1044, 0, 0)
('15', '49')
['Writing & Publishing', 'Storytelling', 'Writing', 'Editorial Writing', 'Journalism', 'Creative Writing', 'Creative', 'Creative Nonfiction']
['Alex Cornacchia', 'Michele Washam', 'Jodi Walker', 'Steve Hoffman', 'Alyssa Demirjian', 'Tony Levelle', 'Kristin Rose', 'Ed Ruggero']
1252230072
True

["The Creative Writer's Toolkit: Writing a Character Study", 'Lindsey Summers']
('Lindsey Summers', 'Writer, Wattpad', 'https://www.skillshare.com/profile/Lindsey-Summers/1443577')
(2010, 0, 4)
('5', '28')
['Writing & Publishing', 'Character Development', 'Writing', 'Fiction', 'Creative Writing', 'Creative', 'Wattpad']
['Lindsey Summers', 'Shesha Taylor', 'Kristen L', 'Lateefah Prise', 'Carmela Rose Aguda', 'Aisha Girl', 'Kathie Rivera', 'Messina Merlo']
1452701491
False
False
07/06/2016
4,123
['[COMPLETE COURSE] Attain The Mindset Of A Best-Selling Author', 'Perrin Briar']
('Perrin Briar', 'Outlaw educator for hire', 'https://www.skillshare.com/user/perrinbriar')
(1351, 0, 0)
('13', '10')
['Writing & Publishing', 'Storytelling', 'Writing', 'Publishing', 'Digital Publishing', 'Editing', 'Creative Writing', 'Creative', 'Creativity']
None
2932419
True
True
07/19/2017
186


In [ ]:
def get_javascript_data(soup):
    projects_section = soup.find_all('script', attrs={"type": "text/javascript"})
    
    javascript = projects_section[1].string
    javascript = javascript.split("SS.serverBootstrap = ", 1)
    
    javascript_data = javascript[1].split(";\n        ")[0]
    javascript_data = json.loads(javascript_data)
    
    return javascript_data

#get author link
def get_title_and_author(soup):
    title = soup.find('title')
    
    return [item.strip() for item in title.text.split('|')]

def get_detial_author_info(javascript_data):
    teacherInfo = javascript_data['pageData']['sectionData']['teacherInfo']
    
    return teacherInfo['fullName'], teacherInfo['headline'], teacherInfo['profileUrl']
    
#find media 
def get_class_description(soup):
    description_tag = soup.find('div', attrs={"class": "about-this-class"}).find('div', attrs={"class": "rich-content-wrapper"})
    
    class_description = ""
    
    images, hyperlinks = 0, 0
    for paragraph in description_tag.findChildren():
        class_description += paragraph.text
        if paragraph.name == 'img':
            images += 1
        elif paragraph.name == 'a':
            hyperlinks += 1
            
    return len(class_description), images, hyperlinks

#extract info
def get_video_length(soup):
    video_content_tag = soup.find('div', attrs={"class": "summary"}).text.strip().strip('\n').strip()
    
    pattern_number = '(\d+)*'
    pattern_length = '(\d+)(?=m)'
    video_num = re.search(pattern_number, video_content_tag).group(0)
    video_length = re.search(pattern_length, video_content_tag).group(0)
    
    return video_num, video_length


# get link
def get_tags(soup):
    tags = []
    tags_section = soup.find('div', attrs={"class": "tags-section"})
    for tag in tags_section.find_all('a'):
        tags.append(tag.text.strip())
        
    return tags
        
def get_project_authors(javascript_data):    
    try:
        return [project['author']['fullName'] for project in javascript_data['pageData']['sectionData']['topProjects']]
    except:
        return None

def get_class_sku(javascript_data):
    return javascript_data['classData']['sku']

def isPremium(javascript_data):
    return javascript_data['pageData']['headerData']['tagText'] == 'Premium class'

def get_start_date(javascript_data):
    return javascript_data['pageData']['syllabusData']['startTs']

def get_enrollment_number(javascript_data):
    return javascript_data['pageData']['sectionData']['numStudents']